# Evaluate your Policy

Now that you have a policy checkpoint, you can easily control your robot with it using methods from ManipulatorRobot and the policy.

## 1. Do inference yourself 

Try this code for running inference for 60 seconds at 30 fps:

In [2]:
# First, initialize the robot
from lerobot.common.robot_devices.robots.configs import KochRobotConfig
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot
from lerobot.common.robot_devices.cameras.configs import OpenCVCameraConfig
from lerobot.common.robot_devices.motors.configs import DynamixelMotorsBusConfig

leader_config = DynamixelMotorsBusConfig(
    port="/dev/tty.usbmodem58760435371",
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl330-m077"),
        "shoulder_lift": (2, "xl330-m077"),
        "elbow_flex": (3, "xl330-m077"),
        "wrist_flex": (4, "xl330-m077"),
        "wrist_roll": (5, "xl330-m077"),
        "gripper": (6, "xl330-m077"),
    },
)

follower_config = DynamixelMotorsBusConfig(
    port="/dev/tty.usbmodem58760433811",
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl430-w250"),
        "shoulder_lift": (2, "xl430-w250"),
        "elbow_flex": (3, "xl330-m288"),
        "wrist_flex": (4, "xl330-m288"),
        "wrist_roll": (5, "xl330-m288"),
        "gripper": (6, "xl330-m288"),
    },
)

robot = ManipulatorRobot(
    KochRobotConfig(
        leader_arms={"main": leader_config},
        follower_arms={"main": follower_config},
        calibration_dir=".cache/calibration/koch",
        cameras={
            "usb": OpenCVCameraConfig(0, fps=30, width=640, height=480),
            "macbook": OpenCVCameraConfig(1, fps=30, width=640, height=480),
        },
    )
)
robot.connect()

Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.
Wrong motor position range detected for wrist_flex. Expected to be in nominal range of [-180, 180] degrees (a full rotation), with a maximum range of [-270, 270] degrees to account for joints that can rotate a bit more, but present value is 356.572265625 degree. This might be due to a cable connection issue creating an artificial 360 degrees jump in motor values. You need to recalibrate by running: `python lerobot/scripts/control_robot.py calibrate`


In [ ]:
from lerobot.common.policies.act.modeling_act import ACTPolicy
import time
from lerobot.scripts.control_robot import busy_wait

inference_time_s = 60
fps = 30
device = "cuda"  # TODO: On Mac, use "mps" or "cpu"

ckpt_path = "outputs/train/act_koch_test/checkpoints/last/pretrained_model"
policy = ACTPolicy.from_pretrained(ckpt_path)
policy.to(device)

for _ in range(inference_time_s * fps):
    start_time = time.perf_counter()

    # Read the follower state and access the frames from the cameras
    observation = robot.capture_observation()

    # Convert to pytorch format: channel first and float32 in [0,1]
    # with batch dimension
    for name in observation:
        if "image" in name:
            observation[name] = observation[name].type(torch.float32) / 255
            observation[name] = observation[name].permute(2, 0, 1).contiguous()
        observation[name] = observation[name].unsqueeze(0)
        observation[name] = observation[name].to(device)

    # Compute the next action with the policy
    # based on the current observation
    action = policy.select_action(observation)
    # Remove batch dimension
    action = action.squeeze(0)
    # Move to cpu, if not already the case
    action = action.to("cpu")
    # Order the robot to move
    robot.send_action(action)

    dt_s = time.perf_counter() - start_time
    busy_wait(1 / fps - dt_s)

In [5]:
robot.disconnect()

## 2. Use our `record` function

Ideally, when controlling your robot with your neural network, you would want to record evaluation episodes and to be able to visualize them later on, or even train on them like in Reinforcement Learning. This pretty much corresponds to recording a new dataset but with a neural network providing the actions instead of teleoperation.

To this end, you can use the record function from lerobot/scripts/control_robot.py but with a policy checkpoint as input. For instance, run this command to record 10 evaluation episodes:

In [ ]:
!python lerobot/scripts/control_robot.py \
  --robot.type=koch \
  --control.type=record \
  --control.fps=30 \
  --control.repo_id=${HF_USER}/eval_act_koch_test \
  --control.tags='["tutorial"]' \
  --control.warmup_time_s=5 \
  --control.episode_time_s=30 \
  --control.reset_time_s=30 \
  --control.num_episodes=10 \
  --control.push_to_hub=true \
  --control.policy.path=outputs/train/act_koch_test/checkpoints/last/pretrained_model

As you can see, it's almost the same command as previously used to record your training dataset. Two things changed:

1. There is an additional `--control.policy.path` argument which indicates the path to your policy checkpoint with (e.g. `outputs/train/eval_koch_test/checkpoints/last/pretrained_model`). You can also use the model repository if you uploaded a model checkpoint to the hub (e.g. `\${HF_USER}/act_koch_test`).

2. The name of dataset begins by eval to reflect that you are running inference (e.g. `\${HF_USER}/eval_act_koch_test`).

## 3. Visualize your evaluation afterwards

You can then visualize your evaluation dataset by running the same command as for visualizing a train dataset but with the new inference dataset as argument:

In [ ]:
!python lerobot/scripts/visualize_dataset.py \
  --repo-id ${HF_USER}/eval_act_koch_test

Congrats, you finished the intro on setting up TaiRo's Koch v1.1, recording a dataset for it, training a policy based on imitation learning and evaluating that policy 🥳

With that you are well equipped to start your own robot experiments. Let's go!! 🚀